# Guia de Melhorias para o Sistema Crypto-Convert

Este notebook apresenta melhorias detalhadas para o sistema crypto-convert-system, fornecendo exemplos práticos de código e boas práticas para cada área de desenvolvimento.

## Estrutura e Organização do Código

Uma boa arquitetura de software é fundamental para a manutenção e escalabilidade do projeto. Vamos explorar como implementar uma estrutura modular usando padrões como MVC.

In [ ]:
# Exemplo de estrutura de diretórios recomendada
'''
crypto-convert-system/
├── src/
│   ├── models/         # Representação de dados e lógica de negócios
│   │   ├── currency.py
│   │   └── transaction.py
│   ├── views/          # Interface com o usuário
│   │   ├── converter_view.py
│   │   └── history_view.py
│   ├── controllers/    # Coordena modelos e views
│   │   └── converter_controller.py
│   ├── services/       # Serviços externos (APIs)
│   │   └── price_service.py
│   └── utils/          # Funções utilitárias
│       ├── formatters.py
│       └── validators.py
├── tests/              # Testes automatizados
├── config/             # Configurações
└── main.py             # Ponto de entrada
'''

### Exemplo de Implementação MVC

Vamos ver como separar responsabilidades usando o padrão MVC:

In [ ]:
# models/currency.py
class Currency:
    def __init__(self, code, name, symbol):
        self.code = code
        self.name = name
        self.symbol = symbol

# models/conversion.py
class ConversionRate:
    def __init__(self, from_currency, to_currency, rate, timestamp):
        self.from_currency = from_currency
        self.to_currency = to_currency
        self.rate = rate
        self.timestamp = timestamp
    
    def convert(self, amount):
        return amount * self.rate

In [ ]:
# controllers/converter_controller.py
class ConverterController:
    def __init__(self, price_service, view):
        self.price_service = price_service
        self.view = view
        
    def handle_conversion(self, from_currency, to_currency, amount):
        try:
            rate = self.price_service.get_rate(from_currency, to_currency)
            result = rate.convert(amount)
            self.view.display_result(result, to_currency)
        except Exception as e:
            self.view.display_error(str(e))
            
    def update_available_currencies(self):
        currencies = self.price_service.get_available_currencies()
        self.view.update_currency_options(currencies)

## Gerenciamento de Estado e Variáveis

Um gerenciamento eficiente do estado da aplicação é crucial para evitar bugs e comportamentos inesperados.

In [ ]:
# Exemplo de classe para gerenciamento de estado
class AppState:
    def __init__(self):
        self._state = {
            "selected_currencies": {
                "from": None,
                "to": None
            },
            "amount": 0,
            "conversion_history": [],
            "is_loading": False,
            "error": None
        }
        self._subscribers = []
    
    def update_state(self, key, value):
        """Atualiza o estado e notifica os assinantes"""
        if key in self._state:
            self._state[key] = value
            self._notify_subscribers()
        else:
            raise KeyError(f"Chave inválida: {key}")
    
    def get_state(self, key=None):
        """Retorna o estado completo ou uma chave específica"""
        if key is None:
            return self._state
        return self._state.get(key)
    
    def subscribe(self, callback):
        """Adiciona um assinante para receber atualizações de estado"""
        if callback not in self._subscribers:
            self._subscribers.append(callback)
        return lambda: self._subscribers.remove(callback)
    
    def _notify_subscribers(self):
        """Notifica todos os assinantes sobre mudanças de estado"""
        for callback in self._subscribers:
            callback(self._state)

In [ ]:
# Exemplo de uso do gerenciador de estado
def main():
    app_state = AppState()
    
    # Função para atualizar a UI quando o estado mudar
    def update_ui(state):
        if state["is_loading"]:
            print("Carregando...")
        elif state["error"]:
            print(f"Erro: {state['error']}")
        else:
            from_currency = state["selected_currencies"]["from"]
            to_currency = state["selected_currencies"]["to"]
            amount = state["amount"]
            if all([from_currency, to_currency, amount]):
                print(f"Convertendo {amount} {from_currency} para {to_currency}")
    
    # Assinar para receber atualizações de estado
    unsubscribe = app_state.subscribe(update_ui)
    
    # Simular interações do usuário
    app_state.update_state("selected_currencies", {"from": "BTC", "to": "USD"})
    app_state.update_state("amount", 1.5)
    app_state.update_state("is_loading", True)
    
    # Simular resultado da API
    app_state.update_state("is_loading", False)
    
    # Limpar assinatura quando não for mais necessária
    unsubscribe()

### Limitando o Escopo de Variáveis

Vamos ver como limitar o escopo de variáveis para evitar efeitos colaterais indesejados:

In [ ]:
# Antes: Variáveis globais
active_currency = "BTC"
exchange_rates = {}

def update_rates():
    global exchange_rates
    # Atualiza as taxas de câmbio
    exchange_rates = {"BTC": 40000, "ETH": 2000}
    
def convert(amount):
    return amount * exchange_rates.get(active_currency, 0)

# Depois: Encapsulamento e escopo limitado
class CurrencyConverter:
    def __init__(self):
        self._active_currency = "BTC"
        self._exchange_rates = {}
    
    def update_rates(self):
        # Taxas de câmbio encapsuladas na classe
        self._exchange_rates = {"BTC": 40000, "ETH": 2000}
    
    def set_active_currency(self, currency):
        if currency in self._exchange_rates:
            self._active_currency = currency
        else:
            raise ValueError(f"Moeda não suportada: {currency}")
    
    def convert(self, amount):
        return amount * self._exchange_rates.get(self._active_currency, 0)

## Tratamento de Erros

Um tratamento de erros abrangente melhora a robustez do sistema e proporciona melhor experiência ao usuário.

In [ ]:
# Definindo exceções personalizadas
class CryptoConvertError(Exception):
    """Classe base para exceções personalizadas no sistema"""
    pass

class ApiConnectionError(CryptoConvertError):
    """Erro de conexão com APIs externas"""
    pass

class InvalidCurrencyError(CryptoConvertError):
    """Moeda não suportada ou inválida"""
    pass

class ConversionError(CryptoConvertError):
    """Erro durante o processo de conversão"""
    pass

# Implementando tratamento de erros
import logging
import requests
from datetime import datetime

# Configuração de logging
logging.basicConfig(
    filename='crypto_convert.log',
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('crypto_convert')

class PriceService:
    def __init__(self, api_key, base_url):
        self.api_key = api_key
        self.base_url = base_url
    
    def get_rate(self, from_currency, to_currency):
        logger.info(f"Solicitando taxa de conversão: {from_currency} -> {to_currency}")
        try:
            url = f"{self.base_url}/convert"
            params = {
                "from": from_currency,
                "to": to_currency,
                "apiKey": self.api_key
            }
            response = requests.get(url, params=params, timeout=5)
            
            if response.status_code == 200:
                data = response.json()
                return ConversionRate(
                    from_currency, 
                    to_currency, 
                    data["rate"], 
                    datetime.now()
                )
            elif response.status_code == 404:
                logger.error(f"Moeda não encontrada: {from_currency} ou {to_currency}")
                raise InvalidCurrencyError(f"Moeda não suportada: {from_currency} ou {to_currency}")
            elif response.status_code == 401:
                logger.error("Chave de API inválida")
                raise ApiConnectionError("Falha de autenticação na API")
            else:
                logger.error(f"Erro na API: {response.status_code}")
                raise ApiConnectionError(f"Falha na requisição: {response.status_code}")
                
        except requests.exceptions.Timeout:
            logger.error("Timeout na conexão com a API")
            raise ApiConnectionError("Tempo de conexão esgotado")
        except requests.exceptions.ConnectionError:
            logger.error("Erro de conexão com a API")
            raise ApiConnectionError("Não foi possível conectar ao servidor")
        except Exception as e:
            logger.error(f"Erro inesperado: {str(e)}")
            raise ConversionError(f"Erro na conversão: {str(e)}")

### Centralizando o Tratamento de Erros

Vamos implementar um middleware para tratar todos os erros de forma consistente:

In [ ]:
# Middleware de tratamento de erros
class ErrorHandler:
    def __init__(self):
        self.logger = logging.getLogger('error_handler')
    
    def handle(self, func):
        """Decorator para tratar erros de funções"""
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except ApiConnectionError as e:
                self.logger.error(f"Erro de API: {str(e)}")
                return {"success": False, "error": "Falha de conexão", "message": str(e)}
            except InvalidCurrencyError as e:
                self.logger.error(f"Moeda inválida: {str(e)}")
                return {"success": False, "error": "Moeda inválida", "message": str(e)}
            except ConversionError as e:
                self.logger.error(f"Erro de conversão: {str(e)}")
                return {"success": False, "error": "Erro de conversão", "message": str(e)}
            except Exception as e:
                self.logger.critical(f"Erro não tratado: {str(e)}")
                return {"success": False, "error": "Erro interno", "message": "Ocorreu um erro inesperado"}
        return wrapper

# Exemplo de uso
error_handler = ErrorHandler()

@error_handler.handle
def perform_conversion(from_currency, to_currency, amount):
    # Simulação de possíveis erros
    if from_currency == "XYZ":
        raise InvalidCurrencyError(f"Moeda não suportada: {from_currency}")
    
    if amount <= 0:
        raise ConversionError("O valor deve ser maior que zero")
    
    # Lógica de conversão
    result = amount * 42  # Valor simulado
    return {"success": True, "result": result, "currency": to_currency}

# Testando diferentes cenários
print(perform_conversion("BTC", "USD", 1))  # Sucesso
print(perform_conversion("XYZ", "USD", 1))  # Erro de moeda inválida 
print(perform_conversion("BTC", "USD", -1)) # Erro de valor inválido

## Performance e Otimização

Melhorar a performance da aplicação é crucial para uma boa experiência do usuário.

In [ ]:
# Implementando cache para dados de API
import time
from functools import lru_cache

class CachedPriceService:
    def __init__(self, api_key, base_url, cache_ttl=300):  # TTL de 5 minutos
        self.api_key = api_key
        self.base_url = base_url
        self.cache_ttl = cache_ttl
        self.rate_cache = {}
        
    def get_rate(self, from_currency, to_currency):
        cache_key = f"{from_currency}_{to_currency}"
        
        # Verificar se o valor está em cache e não expirou
        if cache_key in self.rate_cache:
            cached_data = self.rate_cache[cache_key]
            if time.time() - cached_data["timestamp"] < self.cache_ttl:
                return cached_data["rate"]
        
        # Se chegou aqui, precisa buscar da API
        try:
            # Simulação de chamada de API
            rate = 42.0  # Valor simulado
            
            # Armazenar no cache
            self.rate_cache[cache_key] = {
                "rate": rate,
                "timestamp": time.time()
            }
            
            return rate
        except Exception as e:
            # Se falhar, tenta retornar o cache expirado se disponível
            if cache_key in self.rate_cache:
                return self.rate_cache[cache_key]["rate"]
            raise e

In [ ]:
# Implementando debounce para evitar chamadas excessivas
import threading

def debounce(wait_time):
    """
    Decorator para limitar a frequência de chamadas a uma função
    """
    def decorator(function):
        timer = None
        
        def debounced(*args, **kwargs):
            nonlocal timer
            
            def call_function():
                function(*args, **kwargs)
            
            # Cancelar timer existente
            if timer:
                timer.cancel()
                
            # Configurar novo timer
            timer = threading.Timer(wait_time, call_function)
            timer.start()
            
        return debounced
    return decorator

# Exemplo de uso
@debounce(0.5)  # 500ms
def update_conversion_result(from_currency, to_currency, amount):
    # Esta função só será chamada 500ms após a última chamada
    print(f"Convertendo {amount} {from_currency} para {to_currency}")
    
# Simulação de digitação do usuário em um campo de input
for i in range(5):
    update_conversion_result("BTC", "USD", i)
    time.sleep(0.1)  # Simulando pequenos intervalos entre inputs

# Apenas a última chamada será efetivamente executada
time.sleep(1)  # Esperar para ver o resultado

### Otimização de Chamadas de API em Lote

Para reduzir o número de requisições, podemos implementar chamadas em lote:

In [ ]:
class BatchPriceService:
    def __init__(self, api_key, base_url):
        self.api_key = api_key
        self.base_url = base_url
        self.pending_requests = []
        self.batch_timer = None
        self.batch_interval = 1.0  # segundos
        self.callbacks = {}
        
    def get_rate(self, from_currency, to_currency, callback):
        """
        Adiciona requisição ao lote e programa execução
        """
        request_id = f"{from_currency}_{to_currency}_{time.time()}"
        self.pending_requests.append({
            "id": request_id,
            "from": from_currency,
            "to": to_currency
        })
        self.callbacks[request_id] = callback
        
        # Programa execução do lote se não houver timer ativo
        if not self.batch_timer:
            self.batch_timer = threading.Timer(
                self.batch_interval, 
                self._execute_batch
            )
            self.batch_timer.start()
    
    def _execute_batch(self):
        """
        Executa todas as requisições pendentes em uma única chamada
        """
        if not self.pending_requests:
            self.batch_timer = None
            return
            
        # Cria cópia das requisições pendentes
        requests = self.pending_requests.copy()
        self.pending_requests = []
        self.batch_timer = None
        
        try:
            # Simulação de chamada à API em lote
            print(f"Executando lote com {len(requests)} requisições")
            
            # Em uma implementação real, faria uma única chamada à API
            # com todos os pares de moedas
            
            # Simula resultados
            results = {}
            for req in requests:
                # Simulação de resultado
                rate = 42.0 + (hash(req["from"]) % 100) / 100
                results[req["id"]] = rate
                
            # Notifica com resultados
            for req_id, rate in results.items():
                if req_id in self.callbacks:
                    self.callbacks[req_id](rate)
                    del self.callbacks[req_id]
                    
        except Exception as e:
            print(f"Erro no lote: {str(e)}")
            # Notifica com erro
            for req in requests:
                if req["id"] in self.callbacks:
                    self.callbacks[req["id"]](None, str(e))
                    del self.callbacks[req["id"]]

# Exemplo de uso
service = BatchPriceService("api_key", "https://api.example.com")

def handle_result(rate, error=None):
    if error:
        print(f"Erro: {error}")
    else:
        print(f"Taxa: {rate}")

# Simulando várias requisições simultâneas
service.get_rate("BTC", "USD", handle_result)
service.get_rate("ETH", "USD", handle_result)
service.get_rate("XRP", "USD", handle_result)

# Esperar para ver resultados
time.sleep(2)

## Segurança

Implementar boas práticas de segurança é essencial para proteger seu aplicativo e seus usuários.

In [ ]:
# Sanitização de Entrada
import re
import html

def sanitize_input(input_text):
    """Sanitiza entrada do usuário para prevenir injeção de código"""
    # Remove caracteres especiais
    sanitized = re.sub(r'[^\w\s.,]', '', input_text)
    # Limita tamanho
    sanitized = sanitized[:100]
    # Escapa HTML
    sanitized = html.escape(sanitized)
    return sanitized

# Validação de entrada
def validate_crypto_amount(amount_str):
    """Valida se a entrada é um número positivo"""
    try:
        amount = float(amount_str)
        if amount <= 0:
            return False, "O valor deve ser maior que zero"
        if amount > 1000000:
            return False, "O valor máximo permitido é 1.000.000"
        return True, amount
    except ValueError:
        return False, "Por favor, insira um valor numérico válido"

# Validação de moedas
def validate_currency(currency, allowed_currencies):
    """Valida se a moeda é suportada"""
    if not currency:
        return False, "Selecione uma moeda"
    
    # Sanitiza e padroniza
    currency = sanitize_input(currency).upper()
    
    if currency not in allowed_currencies:
        return False, f"Moeda não suportada: {currency}"
    
    return True, currency

# Exemplo de uso
allowed_currencies = ["BTC", "ETH", "USD", "EUR", "BRL"]

# Teste de validação
test_inputs = [
    ("BTC", "válido"),
    ("btc", "válido após padronização"),
    ("XYZ", "moeda inválida"),
    ("<script>alert('hack')</script>", "tentativa de injeção"),
    ("", "vazio")
]

for currency, description in test_inputs:
    is_valid, result = validate_currency(currency, allowed_currencies)
    if is_valid:
        print(f"✅ {currency} ({description}): {result}")
    else:
        print(f"❌ {currency} ({description}): {result}")

In [ ]:
# Proteção de Dados Sensíveis
import os
from cryptography.fernet import Fernet

class SecureStorage:
    def __init__(self, encryption_key=None):
        """
        Inicializa armazenamento seguro com chave de criptografia
        """
        if encryption_key:
            self.key = encryption_key
        else:
            # Gera nova chave ou carrega existente
            key_file = "secure_storage.key"
            if os.path.exists(key_file):
                with open(key_file, "rb") as f:
                    self.key = f.read()
            else:
                self.key = Fernet.generate_key()
                with open(key_file, "wb") as f:
                    f.write(self.key)
                    
        self.cipher = Fernet(self.key)
    
    def encrypt(self, data):
        """Criptografa dados sensíveis"""
        if isinstance(data, str):
            data = data.encode()
        return self.cipher.encrypt(data)
    
    def decrypt(self, encrypted_data):
        """Descriptografa dados sensíveis"""
        decrypted = self.cipher.decrypt(encrypted_data)
        return decrypted.decode()
    
    def store_api_key(self, service_name, api_key):
        """Armazena chave de API de forma segura"""
        encrypted = self.encrypt(api_key)
        filename = f"{service_name}_api_key.enc"
        with open(filename, "wb") as f:
            f.write(encrypted)
    
    def get_api_key(self, service_name):
        """Recupera chave de API armazenada"""
        filename = f"{service_name}_api_key.enc"
        try:
            with open(filename, "rb") as f:
                encrypted = f.read()
            return self.decrypt(encrypted)
        except FileNotFoundError:
            return None

# Exemplo de uso
# storage = SecureStorage()
# storage.store_api_key("crypto_exchange", "my_super_secret_api_key")
# api_key = storage.get_api_key("crypto_exchange")
# print(f"API Key: {api_key}")

## Testabilidade

A testabilidade é crucial para manter a qualidade do código e facilitar a evolução do sistema.

In [ ]:
# Exemplo de classe com design testável
class CurrencyConverter:
    def __init__(self, price_service):
        """
        Injeção de dependência facilita testes
        """
        self.price_service = price_service
    
    def convert(self, amount, from_currency, to_currency):
        """
        Método com responsabilidade única e sem efeitos colaterais
        """
        if amount <= 0:
            raise ValueError("O valor deve ser maior que zero")
            
        rate = self.price_service.get_rate(from_currency, to_currency)
        converted_amount = amount * rate
        
        return {
            "original": {
                "amount": amount,
                "currency": from_currency
            },
            "converted": {
                "amount": converted_amount,
                "currency": to_currency
            },
            "rate": rate
        }

In [ ]:
# Testes unitários usando unittest
import unittest
from unittest.mock import Mock

class TestCurrencyConverter(unittest.TestCase):
    
    def setUp(self):
        # Mock do serviço de preços
        self.mock_price_service = Mock()
        # Instância do conversor com dependência mockada
        self.converter = CurrencyConverter(self.mock_price_service)
    
    def test_successful_conversion(self):
        # Configuração do mock
        self.mock_price_service.get_rate.return_value = 2.5
        
        # Chamada do método a ser testado
        result = self.converter.convert(10, "BTC", "USD")
        
        # Verificações
        self.mock_price_service.get_rate.assert_called_once_with("BTC", "USD")
        self.assertEqual(result["original"]["amount"], 10)
        self.assertEqual(result["original"]["currency"], "BTC")
        self.assertEqual(result["converted"]["amount"], 25)
        self.assertEqual(result["converted"]["currency"], "USD")
        self.assertEqual(result["rate"], 2.5)
    
    def test_negative_amount(self):
        # Teste de validação de entrada
        with self.assertRaises(ValueError):
            self.converter.convert(-5, "BTC", "USD")
    
    def test_zero_amount(self):
        # Teste de validação de entrada
        with self.assertRaises(ValueError):
            self.converter.convert(0, "BTC", "USD")
    
    def test_price_service_error(self):
        # Simula erro no serviço de preços
        self.mock_price_service.get_rate.side_effect = Exception("API Error")
        
        # Verifica se a exceção é propagada
        with self.assertRaises(Exception):
            self.converter.convert(10, "BTC", "USD")

# Para executar os testes
# unittest.main()

### Separando Lógica de Negócios e UI

Vamos ver como separar a lógica de negócios da interface, facilitando testes:

In [ ]:
# Separação de lógica de negócios e UI

# Lógica de negócios (fácil de testar)
class ConversionLogic:
    def __init__(self, price_service):
        self.price_service = price_service
        
    def validate_input(self, amount, from_currency, to_currency):
        errors = []
        
        try:
            amount = float(amount)
            if amount <= 0:
                errors.append("O valor deve ser maior que zero")
        except (ValueError, TypeError):
            errors.append("Por favor, insira um valor numérico válido")
            
        if not from_currency:
            errors.append("Selecione a moeda de origem")
            
        if not to_currency:
            errors.append("Selecione a moeda de destino")
            
        return len(errors) == 0, errors, amount
    
    def get_conversion_result(self, amount, from_currency, to_currency):
        is_valid, errors, parsed_amount = self.validate_input(
            amount, from_currency, to_currency)
            
        if not is_valid:
            return {
                "success": False,
                "errors": errors
            }
            
        try:
            rate = self.price_service.get_rate(from_currency, to_currency)
            converted_amount = parsed_amount * rate
            
            return {
                "success": True,
                "result": {
                    "original": {
                        "amount": parsed_amount,
                        "currency": from_currency
                    },
                    "converted": {
                        "amount": converted_amount,
                        "currency": to_currency
                    },
                    "rate": rate
                }
            }
        except Exception as e:
            return {
                "success": False,
                "errors": [str(e)]
            }

# Interface (interação com o usuário)
class ConversionUI:
    def __init__(self, conversion_logic):
        self.conversion_logic = conversion_logic
        
    def handle_convert_button_click(self, amount_input, from_select, to_select, result_element):
        # Na interface real, estes seriam elementos da UI
        amount = amount_input.value
        from_currency = from_select.value
        to_currency = to_select.value
        
        # Chama a lógica de negócios
        response = self.conversion_logic.get_conversion_result(
            amount, from_currency, to_currency)
            
        # Atualiza a UI com o resultado
        if response["success"]:
            result = response["result"]
            original = result["original"]
            converted = result["converted"]
            
            result_element.innerHTML = (
                f"{original['amount']} {original['currency']} = "
                f"{converted['amount']} {converted['currency']}"
            )
        else:
            error_message = "<br>".join(response["errors"])
            result_element.innerHTML = f"<span class='error'>{error_message}</span>"

## Acessibilidade e UX

Melhorar a acessibilidade e experiência do usuário torna seu aplicativo mais inclusivo e amigável.

In [ ]:
# Exemplo de HTML com melhorias de acessibilidade
html_example = """
<form aria-labelledby="form-title" class="conversion-form">
    <h2 id="form-title">Conversor de Criptomoedas</h2>
    
    <div class="form-group">
        <label for="amount" id="amount-label">Valor:</label>
        <input 
            type="number" 
            id="amount" 
            name="amount" 
            min="0.00001" 
            step="any" 
            required 
            aria-labelledby="amount-label"
            aria-describedby="amount-help"
        >
        <small id="amount-help" class="helper-text">
            Insira o valor que deseja converter
        </small>
    </div>
    
    <div class="form-group">
        <label for="from-currency" id="from-label">De:</label>
        <select 
            id="from-currency" 
            name="from-currency" 
            required 
            aria-labelledby="from-label"
        >
            <option value="">Selecione a moeda</option>
            <option value="BTC">Bitcoin (BTC)</option>
            <option value="ETH">Ethereum (ETH)</option>
            <!-- Outras opções -->
        </select>
    </div>
    
    <div class="form-group">
        <label for="to-currency" id="to-label">Para:</label>
        <select 
            id="to-currency" 
            name="to-currency" 
            required 
            aria-labelledby="to-label"
        >
            <option value="">Selecione a moeda</option>
            <option value="USD">Dólar (USD)</option>
            <option value="EUR">Euro (EUR)</option>
            <!-- Outras opções -->
        </select>
    </div>
    
    <button 
        type="submit" 
        aria-label="Converter valor"
        class="convert-button"
    >
        Converter
    </button>
    
    <div 
        id="result" 
        class="result-container" 
        aria-live="polite"
        role="status"
    >
        <!-- Resultados serão exibidos aqui -->
    </div>
    
    <div 
        id="loading-indicator" 
        class="loading-indicator" 
        aria-hidden="true"
        hidden
    >
        <span class="sr-only">Carregando...</span>
        <div class="spinner"></div>
    </div>
</form>
"""

In [ ]:
# Exemplo de função para fornecer feedback durante operações assíncronas

def setup_ui_feedback():
    """
    Configurar UI para fornecer feedback adequado durante operações assíncronas
    """
    # Em uma aplicação real, esto seria integrado com o framework de UI
    
    # Simulação dos elementos da UI
    class UIElement:
        def __init__(self, id):
            self.id = id
            self.hidden = True
            self.disabled = False
            self.aria_busy = False
            self.innerHTML = ""
            
        def setAttribute(self, attr, value):
            setattr(self, attr, value)
            
        def removeAttribute(self, attr):
            setattr(self, attr, None)
            
    # Elementos simulados
    form = UIElement("conversion-form")
    button = UIElement("convert-button")
    loading = UIElement("loading-indicator")
    result = UIElement("result")
    
    # Função para mostrar estado de carregamento
    def show_loading_state():
        button.disabled = True
        button.innerHTML = "Processando..."
        form.setAttribute("aria-busy", "true")
        loading.hidden = False
        loading.setAttribute("aria-hidden", "false")
        result.innerHTML = "Calculando conversão..."
        
    # Função para esconder estado de carregamento
    def hide_loading_state():
        button.disabled = False
        button.innerHTML = "Converter"
        form.removeAttribute("aria-busy")
        loading.hidden = True
        loading.setAttribute("aria-hidden", "true")
    
    # Função assíncrona de conversão
    async def perform_conversion(from_currency, to_currency, amount):
        # Mostra indicadores de carregamento
        show_loading_state()
        
        try:
            # Simula uma operação assíncrona (ex: chamada de API)
            await asyncio.sleep(2)  # Simula atraso de rede
            
            # Resultado simulado
            conversion_rate = 42000
            converted_amount = amount * conversion_rate
            
            # Atualiza resultado com sucesso
            result.innerHTML = (
                f"{amount} {from_currency} = "
                f"{converted_amount} {to_currency}"
            )
            
        except Exception as e:
            # Atualiza resultado com erro
            result.innerHTML = f"Erro: {str(e)}"
            
        finally:
            # Sempre esconde indicadores de carregamento
            hide_loading_state()
    
    # Retorna funções úteis
    return {
        "show_loading": show_loading_state,
        "hide_loading": hide_loading_state,
        "convert": perform_conversion
    }

# Para uso em uma aplicação real:
# ui_handler = setup_ui_feedback()
# await ui_handler["convert"]("BTC", "USD", 1.0)

## Gerenciamento de Dependências

Técnicas eficientes de gerenciamento de dependências melhoram a performance e manutenção do sistema.

In [ ]:
# Implementação de lazy loading
class LazyLoader:
    """
    Carrega módulos e componentes apenas quando necessário
    """
    def __init__(self):
        self._loaded_modules = {}
        
    def get_module(self, module_name):
        """
        Carrega um módulo apenas quando solicitado pela primeira vez
        """
        if module_name not in self._loaded_modules:
            print(f"Carregando módulo: {module_name}")
            
            # Simula carregamento de diferentes módulos
            if module_name == "chart_module":
                # Em uma aplicação real, importaria o módulo
                # import heavy_chart_library as chart_module
                self._loaded_modules[module_name] = {
                    "create_chart": lambda data: f"Chart created with {len(data)} points"
                }
                
            elif module_name == "export_module":
                self._loaded_modules[module_name] = {
                    "export_to_csv": lambda data: f"CSV with {len(data)} rows",
                    "export_to_pdf": lambda data: f"PDF with {len(data)} rows"
                }
                
            elif module_name == "history_module":
                self._loaded_modules[module_name] = {
                    "get_history": lambda: "Transaction history loaded"
                }
                
        return self._loaded_modules[module_name]

# Exemplo de App utilizando lazy loading
class CryptoApp:
    def __init__(self):
        self.loader = LazyLoader()
        
    def convert_currency(self, amount, from_currency, to_currency):
        # Funcionalidade principal, carregada imediatamente
        print(f"Converting {amount} {from_currency} to {to_currency}")
        return amount * 42000
        
    def show_chart(self, currency, timeframe):
        # Carrega módulo de gráficos sob demanda
        chart_module = self.loader.get_module("chart_module")
        data = [random.random() for _ in range(100)]  # Dados simulados
        return chart_module["create_chart"](data)
    
    def export_data(self, format_type, data):
        # Carrega módulo de exportação sob demanda
        export_module = self.loader.get_module("export_module")
        
        if format_type == "csv":
            return export_module["export_to_csv"](data)
        elif format_type == "pdf":
            return export_module["export_to_pdf"](data)
        else:
            raise ValueError(f"Formato não suportado: {format_type}")
    
    def show_history(self):
        # Carrega módulo de histórico sob demanda
        history_module = self.loader.get_module("history_module")
        return history_module["get_history"]()

# Demonstração
app = CryptoApp()

# Primeira funcionalidade (carregada imediatamente)
result = app.convert_currency(1, "BTC", "USD")
print(f"Result: {result}")

# Segunda funcionalidade (carrega módulo chart_module sob demanda)
chart = app.show_chart("BTC", "1d")
print(f"Chart: {chart}")

# Terceira funcionalidade (carrega módulo export_module sob demanda)
export = app.export_data("csv", [1, 2, 3])
print(f"Export: {export}")

In [ ]:
# Minimizando dependências externas

# Antes: Dependências excessivas
"""
import pandas as pd  # Para processamento de dados
import numpy as np   # Para cálculos
import matplotlib.pyplot as plt  # Para visualização
import seaborn as sns  # Para visualização aprimorada
import requests  # Para API
import beautiful_soup  # Para parsing
import json  # Para processamento JSON

def process_data(data):
    df = pd.DataFrame(data)
    df['moving_avg'] = df['price'].rolling(window=7).mean()
    return df

def analyze_trend(df):
    # Usar numpy para cálculos
    returns = np.diff(df['price']) / df['price'][:-1]
    volatility = np.std(returns)
    return volatility

def visualize(df):
    # Usar matplotlib e seaborn para visualização
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x='date', y='price')
    sns.lineplot(data=df, x='date', y='moving_avg', color='red')
    plt.title('Price Trend')
    plt.show()
"""

# Depois: Dependências minimizadas e isoladas

class CryptoAnalyzer:
    def __init__(self):
        self._visualization_module = None
        
    def process_data(self, data):
        """Processamento básico sem dependências externas"""
        processed_data = []
        
        # Implementação manual de média móvel
        window_size = 7
        
        for i, entry in enumerate(data):
            processed_entry = dict(entry)  # Copia o dicionário
            
            # Calcula média móvel manualmente
            if i >= window_size - 1:
                window = [data[j]['price'] for j in range(i - window_size + 1, i + 1)]
                moving_avg = sum(window) / window_size
                processed_entry['moving_avg'] = moving_avg
            else:
                processed_entry['moving_avg'] = None
                
            processed_data.append(processed_entry)
            
        return processed_data
    
    def analyze_trend(self, data):
        """Análise de tendência sem dependências externas"""
        if len(data) < 2:
            return 0
            
        # Calcula retornos manualmente
        returns = []
        for i in range(1, len(data)):
            prev_price = data[i-1]['price']
            curr_price = data[i]['price']
            return_val = (curr_price - prev_price) / prev_price
            returns.append(return_val)
            
        # Calcula volatilidade (desvio padrão) manualmente
        mean_return = sum(returns) / len(returns)
        squared_diffs = [(r - mean_return) ** 2 for r in returns]
        variance = sum(squared_diffs) / len(returns)
        volatility = variance ** 0.5  # Raiz quadrada da variância
        
        return volatility
    
    def visualize(self, data):
        """
        Carrega o módulo de visualização apenas se necessário
        """
        if not self._visualization_module:
            try:
                # Importa dependências apenas quando necessário
                import matplotlib.pyplot as plt
                import seaborn as sns
                
                self._visualization_module = {
                    'plt': plt,
                    'sns': sns
                }
            except ImportError:
                print("Bibliotecas de visualização não disponíveis")
                return "Não foi possível criar visualização: bibliotecas ausentes"
                
        # Usa as bibliotecas importadas
        plt = self._visualization_module['plt']
        sns = self._visualization_module['sns']
        
        # Prepara dados para visualização
        dates = [entry['date'] for entry in data]
        prices = [entry['price'] for entry in data]
        moving_avgs = [entry.get('moving_avg') for entry in data]
        
        # Cria visualização
        plt.figure(figsize=(10, 6))
        plt.plot(dates, prices, label='Preço')
        plt.plot(dates, moving_avgs, color='red', label='Média Móvel (7d)')
        plt.title('Tendência de Preço')
        plt.legend()
        
        # Em um aplicativo real, você pode salvar a imagem ou retorná-la
        plt.savefig('price_trend.png')
        plt.close()
        
        return "Visualização criada com sucesso: price_trend.png"

## Conclusão

Neste notebook, exploramos diversas técnicas para melhorar a qualidade e manutenção do sistema crypto-convert-system:

1. **Estrutura e Organização do Código**: Implementamos uma arquitetura modular usando o padrão MVC.
2. **Gerenciamento de Estado**: Demonstramos como criar um gerenciador de estado centralizado e limitar o escopo de variáveis.
3. **Tratamento de Erros**: Exploramos estratégias para tratamento de erros abrangente e logging consistente.
4. **Performance e Otimização**: Implementamos técnicas como cache, debounce e otimização de chamadas de API em lote.
5. **Segurança**: Aprendemos como sanitizar entradas e proteger dados sensíveis.
6. **Testabilidade**: Demonstramos como separar a lógica de negócios da UI para facilitar testes.
7. **Acessibilidade e UX**: Implementamos práticas recomendadas para acessibilidade e feedback do usuário.
8. **Gerenciamento de Dependências**: Exploramos o lazy loading e como minimizar dependências externas.

Aplicando estas técnicas, você pode transformar o sistema crypto-convert-system em uma aplicação robusta, eficiente e fácil de manter.